In [ ]:
from data_process.nnlqp.dataset import get_torch_data
from data_process.nnlqp.feature.graph_feature import parse_from_onnx
import os
import numpy as np
import onnx 

root_dir = './datasets/nnlqp/unseen_structure'
gt_file = './datasets/nnlqp/unseen_structure/gt.txt'

# 1. 读取 gt.txt 内容
with open(gt_file, 'r') as f:
    dataset = f.readlines()


import numpy as np
from data_process.nnlqp.feature.feature_utils import OPS
def get_adj_and_op(nx_G):
    # 1. 获取所有节点名称并排序（确保顺序一致）
    node_list = sorted(nx_G.nodes)
    node2idx = {node_name: idx for idx, node_name in enumerate(node_list)}

    num_nodes = len(node_list)
    num_ops = max(op["code"] for op in OPS.values()) + 1  # +1 防止漏掉最大 code

    # 2. 初始化邻接矩阵 A 和特征矩阵 X
    A = np.zeros((num_nodes, num_nodes), dtype=np.float32)
    X = np.zeros((num_nodes, num_ops), dtype=np.float32)

    # 3. 填充节点特征矩阵 X
    for node in nx_G.nodes.data():
        node_name = node[0]
        attrs = node[1]["attr"].attributes
        op_type = attrs["type"]
        op_code = OPS[op_type]["code"] if op_type in OPS else 0
        node_idx = node2idx[node_name]
        X[node_idx, op_code] = 1.0

    # 4. 填充邻接矩阵 A（有向图：从 src 指向 tgt）
    for src, tgt in nx_G.edges():
        src_idx = node2idx[src]
        tgt_idx = node2idx[tgt]
        A[src_idx, tgt_idx] = 1.0

    # 5. 打印确认
    print("A shape (adj):", A.shape)       # [n_nodes, n_nodes]
    print("X shape (one-hot):", X.shape)   # [n_nodes, num_ops]
    graph, feature = A, X
    return graph, feature

root_dir = './datasets/nnlqp/unseen_structure'
all_tokens = []
all_stat_features = []
for i, line in enumerate(dataset):
    print(f"{i}/{len(dataset)}")
    line = line.rstrip()
    items = line.split(" ")
    speed_id = str(items[0])
    graph_id = str(items[1])
    batch_size = int(items[2])
    cost_time = float(items[3])
    plt_id = int(items[5])
    onnx_path = os.path.join(root_dir, graph_id)
    try:
        # 2. 解析 ONNX
        GG = onnx.load(onnx_path)
        data, sf = get_torch_data(GG, batch_size, cost_time, 'nerf') #process onnx file，
        tokens = data['netcode']
        all_tokens.append(tokens)
        all_stat_features.append(sf)
        print(tokens.shape, sf.shape)
    except Exception as e:
        print(f"[Warning] Skipped {onnx_path} due to: {e}")
    
    # break

0/20000
 - Warning: onnx version should in ['1.7.0', '1.8.1'], but with 1.18.0, infer shape may cause error
 - Warning: onnx version should in ['1.7.0', '1.8.1'], but with 1.18.0, infer shape may cause error
torch.Size([23, 960]) torch.Size([256])
1/20000
 - Warning: onnx version should in ['1.7.0', '1.8.1'], but with 1.18.0, infer shape may cause error
 - Warning: onnx version should in ['1.7.0', '1.8.1'], but with 1.18.0, infer shape may cause error
torch.Size([23, 960]) torch.Size([256])
2/20000
 - Warning: onnx version should in ['1.7.0', '1.8.1'], but with 1.18.0, infer shape may cause error
 - Warning: onnx version should in ['1.7.0', '1.8.1'], but with 1.18.0, infer shape may cause error
torch.Size([23, 960]) torch.Size([256])
3/20000
 - Warning: onnx version should in ['1.7.0', '1.8.1'], but with 1.18.0, infer shape may cause error
 - Warning: onnx version should in ['1.7.0', '1.8.1'], but with 1.18.0, infer shape may cause error
torch.Size([23, 960]) torch.Size([256])
4/20000


In [ ]:
import pickle 
with open('./datasets/nnlqp/unseen_structure/graph/all_tokens.pkl', 'wb') as f:
    pickle.dump(all_tokens, f)
with open('./datasets/nnlqp/unseen_structure/graph/all_stat_features.pkl', 'wb') as f:
    pickle.dump(all_stat_features, f)


A shape (adj): (21, 21)
X shape (one-hot): (21, 29)
